# MD of Lennard-Jones Particles

In this exercise, we implement an MD code for Lennard-Jones particles.
We choose liquid Argon as a test system.
We start with $N=n^3$ atoms placed on a cubic lattice and with initial velocities chosen from a random velocity distribution corresponding to an initial temperature of $T=87\,\mathsf{K}$.

Throughout this exercise you will be asked to fill in blanks in the code.

In [ ]:
import itertools

import matplotlib.pyplot as plt
import numba
import numpy as np
import scipy.constants as const
import tqdm

plt.style.use("ggplot")

We start by assigning some constants.
`epsilon` and `sigma` are the LJ parameters for Argon, `m` is the mass.
We will initialize velocities to a temperature `temp`.
`density` contains the liquid phase density of Argon at the specified temperature.

Please compute the value of $k_\mathsf{B}T$ in molecular units.

In [ ]:
epsilon = 0.9977  # kJ/mol
sigma = 0.34  # nm
m = 39.95  # amu
temp = 87.0  # K
density = 1.3973  # g/cm^3
density *= (const.gram / const.u) * (const.centi / const.nano) ** (-3)  # amu/nm^3

kt = ...  # kJ/mol

# If you get an `AssertionError`, your result is wrong. Also, please don't copy the result ;)
assert np.isclose(kt, 0.7233582477793319)

As before, we need to be able to compute the potential and the force.
Complete the definition of the following function.
It should return the value of the LJ-Potential at the specified distance.

$$
V(r) = 4\varepsilon\left[\left(\frac{\sigma}{r}\right)^{12}-\left(\frac{\sigma}{r}\right)^{6}\right]
$$

In [ ]:
@numba.njit()
def potential(r, epsilon, sigma):
    """
    Computes the value of the LJ potential at r, given the well depth epsilon and
    the contact distance sigma.
    """
    ...
    return epot

Verify that your implementation looks reasonable.

In [ ]:
r = np.linspace(0.95 * sigma, 3.0 * sigma, 1000)
v = potential(r, epsilon=epsilon, sigma=sigma)


plt.figure()
plt.plot(r, v)
plt.show()

Next, derive an expression for the force and implement it below.

In [ ]:
@numba.njit()
def force(r, epsilon, sigma):
    """
    Computes the magnitude of the LJ force at r, given the well depth epsilon and
    the contact distance sigma.
    """
    ...
    return f

The following plot should give two (almost) identical force profiles. The second one corresponds to the numerical derivative of your potential energy function. If the two curves match, your implementation is correct.

In [ ]:
f = force(r, epsilon=epsilon, sigma=sigma)
ftest = -np.gradient(v, r)

plt.figure()
plt.plot(r, f)
plt.plot(r, ftest)
plt.show()

Initially, we would like to place atoms on a cubic lattice.
We will work with $6^3$ atoms.
Let's start by computing the volume of a cube containing this many atoms.
Next, compute the length of each vertex of the cube.

In [ ]:
n = 6
natoms = n**3
volume = ...  # nm^3
box = ...  # nm

Now that we know the size of our simulation cell, we try to implement the code that puts atoms on a lattice.
Fill in the blanks.

In [ ]:
@numba.njit()
def generate_cubic_lattice(n, box):
    """
    Puts n**3 atoms on an n x n x n cubic lattice.
    Coordinates are scaled to lie within [0, box).
    """
    natoms = n**3
    lattice = np.zeros((natoms, 3))
    atom = 0
    for i in range(n):
        for j in range(n):
            for k in range(n):
                lattice[atom, 0] = ...
                lattice[atom, 1] = ...
                lattice[atom, 2] = ...
                atom += 1
    lattice *= box / n
    return lattice

Verify that your implementation is correct by executing the following cell.
In here, we construct a cubic lattice for $6^3$ atoms in a volume corresponding to the density specified above.

In [ ]:
def plot_frame(x, box, v=None):
    """
    Makes a 3D scatter plot of a position array x.
    Also plots a cubic wireframe with edge length box.
    """
    fig = plt.figure()
    ax = fig.add_subplot(projection="3d")
    # Plot the box frame.
    r = [0, 1]
    for s, e in itertools.combinations(itertools.product(r, r, r), 2):
        s = np.array(s)
        e = np.array(e)
        if np.sum(np.abs(e - s)) == 1:
            X, Y, Z = np.vstack([s, e]).T * box
            ax.plot3D(X, Y, Z, color="black", alpha=0.5)

    X, Y, Z = x.T
    ax.scatter3D(X, Y, Z)

    if v is not None:
        VX, VY, VZ = v.T
        ax.quiver(X, Y, Z, VX, VY, VZ)
    plt.show()


x0 = generate_cubic_lattice(n, box)
plot_frame(x0, box)

We also need initial velocities.
We have done this before.
Creat an array `v0` with shape `(natoms, 3)` random values drawn from a normal distribution.
Multiply by $\sqrt{k_\mathsf{B}T/m}$ to obtain a velocity distribution.

In [ ]:
v0 = ...

In the following, we plot the velocities as a vector field. The arrows should be small compared to the box size if you did everything right.

In [ ]:
plot_frame(x0, box, v=v0)

The next part is the trickiest part of the algorithm.
We need to evaluate the forces acting between each pair of atoms.
These forces need to be summed up for each atom in the system.
We make use of Newton's third law in this code.
We also consider the minimum image convention.
Fill in the blanks.

In [ ]:
@numba.njit()
def forces_and_potential(x, epsilon, sigma):
    """
    Computes the total potential energy and the forces over all pairs of atoms.
    """
    natoms = x.shape[0]
    epot = 0.0
    f = np.zeros_like(x)

    for i in range(natoms - 1):
        for j in range(i + 1, natoms):
            dx = ...  # Compute the distance vector.
            dx -= np.round(dx / box) * box  # This is the minimum image convention in a concise form.
            r = ... # Compute the norm of the distance vector.

            fij = ...  # Compute the magnitude of the force between the atoms i and j.
            u = dx / r
            f[i] ?= fij * u  # Replace ? with either + or -. Determine the sign by considering Newton's third law.
            f[j] ?= fij * u  # Same here.

            epot += ...  # Compute the potential.

    return f, epot

Now, complete the definition of the following function for the computation of the kinetic energy.

In [ ]:
def kinetic(v, m):
    """
    Computes the kinetic energy.
    """
    ...
    return ekin

That's it, now we can write our MD code. Fill in the blanks.

In [ ]:
def md(nsteps, dt, x0, v0, m, box, epsilon, sigma):
    """
    Performs VV-MD for a LJ system.
    """
    x = np.copy(x0)
    v = np.copy(v0)
    f, epot = forces_and_potential(x, epsilon, sigma)
    ekin = kinetic(v, m)

    trj_x = [np.copy(x)]
    trj_epot = [epot]
    trj_ekin = [ekin]

    for i in tqdm.trange(nsteps):
        x += ...
        v += ...
        f, epot = ...
        v += ...
        ekin = ...

        trj_x.append(np.copy(x))
        trj_epot.append(epot)
        trj_ekin.append(ekin)

    t = np.arange(0, nsteps + 1) * dt
    trj_x = np.array(trj_x)
    trj_epot = np.array(trj_epot)
    trj_ekin = np.array(trj_ekin)

    return t, trj_x, trj_epot, trj_ekin

Here, we run a short MD.

In [ ]:
nsteps = 5000
dt = 0.001
t, x, epot, ekin = md(
    nsteps=nsteps,
    dt=dt,
    x0=x0,
    v0=v0,
    m=m,
    box=box,
    epsilon=epsilon,
    sigma=sigma,
)

Verify energy conservation.

In [ ]:
etot = epot + ekin
epot -= np.mean(epot)
ekin -= np.mean(ekin)
etot -= np.mean(etot)
plt.plot(t, epot)
plt.plot(t, ekin)
plt.plot(t, etot)

Visualize selected snapshots.

In [ ]:
plot_frame(x[10], box=box)